# Assignment 1

Link to github: https://github.com/NikolajT84/CSS_assignment1

### Collaboration

We have written the code working together on Nikolaj's machine, which is why all commits are from the same user.

In [1]:
# Imports for all exercises
import requests
from bs4 import BeautifulSoup
import json

### Part 1

We follow the instructions, and use bs4 to parse the html.

In [5]:
# Get the HTML content
link = 'https://ic2s2-2023.org/program'
r = requests.get(link)
soup = BeautifulSoup(r.content)

# Find all the lists of things going on
programs = soup.find_all("ul", {"class":"nav_list"})

# Find all lists of authors
program_authors = [program.find_all("i") for program in programs]

# Get the text
all_authors = [author.text for authors in program_authors for author in authors]

# Get the individual names
all_names = [name.lower() for authors in all_authors for name in authors.split(", ")]
all_names_unique = list(set(all_names))

print(len(all_names_unique))
print(all_names_unique[:100])

# Save file as json
with open('authors.json', 'w') as f:
    json.dump(all_names_unique, f)

1472
['kristoffer lind glavind', 'matthew deverna', 'sho cho', 'balazs vedres', 'manju bura', 'anya hommadova lu', 'sagar kumar', 'piotr bródka', 'laura maria alessandretti', 'salvatore giorgi', 'naoki yoshinaga', 'ashlyn b. aske', 'nakao ran', 'sanja scepanovic', 'camille testard', 'sharon kang', 'alexandra segerberg', 'angelo brayner', 'michael cook', 'shaun bevan', 'laura boeschoten', 'yuan zhang', 'indrajeet patil', 'michael szell', 'daniele rama', 'anna rogers', 'neeley pate', 'rob chew', 'jon roozenbeek', 'louis boucherie', 'kai-cheng yang', 'xindi wang', 'brenda curtis', 'ryan louis stevens', 'nalette brodnax', 'andrea failla', 'miriam hurtado bodell', 'daniel larremore', 'lisette espin-noboa', 'zishan lan', 'edmond awad', 'vadim voskresenskii', 'yu-wen chen', 'katayoun farrahi', 'zoe k. rahwan', 'dan dai', 'ivano bison', 'd. sunshine hillygus', 'filipi nascimento silva', 'miriam redi', 'zhemeng xie', 'gemma read', 'gerardo iñiguez', 'cynthia rudin', 'yuan liao', 'fredrik jansso

5. How many unique researches do you get?

As seen in the code output above we get 1472 unique authors.

6. Explain the process you followed to web-scrape the page. Which choices did you make to accurately retrieve as many names as possible? Which strategies did you use to assess the quality of your final list? Explain your reasoning and your choices?

We first go to the webpage to inspect the url. We notice that all the programs are in elements marked 'ul' with class name 'nav_list'. We get these programs and find all the names in them, contained in the elements marked 'i'. Then it's just a matter of formatting the resulting lists and deleting duplicates. The approach of taking all the navigable lists ensures that we get names from all the types of events. We look at a subset of the names, to see that it is indeed names and not something else we have found.

### Part 2

1. What are pros and cons of the custom-made data used in Centola's experiment (the first study presented in the lecture) and the ready-made data used in Nicolaides's study (the second study presented in the lecture)? You can support your arguments based on the content of the lecture and the information you read in Chapter 2.3 of the book.

For the custom-made data used by Centola, there are the usual downsides to this type of data: it probably took quite a lot of effort to set up and design the experiment, and they couldn't be sure beforehand how many people they would get to sign up to the network. On the other hand, they got the exact, specific data they wanted, and were thus able to get more out of their limited data.

For the ready-made data used in the second study, there was the advantage that all the data was collected beforehand. Thus, the researchers could jump right to analyzing it. The volume of data from the fitness tracker would most likely also be way bigger than anything they could have designed themselves. The fact that data is collected over time also provides the added advantage of facilitating longitudinal analysis. The downside is, that since they had no control over the collection process, they had to be more careful when doing their causal inference, as some variables could not be measured directly and the biases of the collection process is harder to uncover.

2. How do you think these differences can influence the interpretation of the results in each study?

With ready-made data, one will often have to do some data-manipulation to infer the variables of interest. This can introduce added uncertainty to the conclusions drawn from the study. On the other hand, with custom-made data, one will often have to extrapolate/do inference from fewer data-points, since the collection process is more ardous. This can limit how value of the study's conclusions.



### Part 3

